# Imports

In [1]:
from pathlib import Path

import pandas as pd

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

from mlops.pipelines.churn import ChurnPipeline

from mlops.config import MODEL_CONFIG

In [2]:
config = MODEL_CONFIG["models"]["churn"]

## Data

In [3]:
data_path = Path("../data") / "churn" / "data.parquet"
dataset = pd.read_parquet(data_path)

In [4]:
X = dataset.drop(columns=[config["target"]])
y = dataset[config["target"]]

numeric_features = [
    X.columns.get_loc(c)
    for c in X.select_dtypes(include=["int64", "float64"])
    .drop(config["drop_features"]["numerical"], axis=1)
    .columns.values
]

categorical_features = [
    X.columns.get_loc(c)
    for c in X.select_dtypes(include=["object"])
    .drop(config["drop_features"]["categorical"], axis=1)
    .columns.values
]

## Pipeline


In [5]:
pipeline = ChurnPipeline(params={}, numeric_features=numeric_features, categorical_features=categorical_features).build()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=config["test_size"], random_state=config["random_seed"]
)

In [7]:
pipeline.fit(X_train, y_train)

/Users/migduroli/dev/loyola/mlops/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/migduroli/dev/loyola/mlops/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/migduroli/dev/loyola/mlops/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/migduroli/dev/loyola/mlops/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('outlier_clipper',
                                                                                          OutlierClipper()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         [3, 6,
                                                                          7, 8,
                                                                          9, 10,
                                                                          11,
                                                                          12]),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         [4,
                                                                          5])])),
                                       ('mlp_classifier',
                                        MLPClassifier(random_state=123456))]),
             n_jobs=-1, param_grid={}, refit='roc_auc',
             scoring=['accuracy', 'f1', 'roc_auc'])

## Prediction

In [8]:
predictions, probabilities = pipeline.predict(X_test), pipeline.predict_proba(X_test)

In [9]:
results = pd.DataFrame(
    {"actual": y_test, "prediction": predictions, "probability": probabilities[:, 1]}  # Probability of positive class
)

In [10]:
results

,actual,prediction,probability
8369,1,0,0.084875
5395,0,1,0.542969
7996,0,0,0.031986
4157,0,0,0.018895
3324,0,0,0.279897
...,...,...,...
8336,1,0,0.451282
8221,1,1,0.982396
7049,1,0,0.148617
986,0,0,0.091119
